In [1]:
import tensorflow as tf 

C:\Users\Lenovo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Model Definition:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])


C:\Users\Lenovo\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [3]:
#Data Loading and Preprocessing:

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
import mpi4py
print(mpi4py.get_config())

{'define_macros': 'MSMPI_VER=0xa01', 'include_dirs': 'C:\\Program Files (x86)\\Microsoft SDKs\\MPI\\Include', 'libraries': 'msmpi', 'library_dirs': 'C:\\Program Files (x86)\\Microsoft SDKs\\MPI\\Lib\\x64'}


In [5]:
#MPI Initialization:

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [6]:
#Training Function:

def train(model, x_train, y_train, rank, size):
    n = len(x_train)
    chunk_size = n // size
    start = rank * chunk_size
    end = (rank + 1) * chunk_size if rank != size - 1 else n
    
    x_train_chunk = x_train[start:end]
    y_train_chunk = y_train[start:end]
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train_chunk, y_train_chunk, epochs=1, batch_size=32)
    
    train_loss, train_acc = model.evaluate(x_train_chunk, y_train_chunk, verbose=2)
    train_acc = comm.allreduce(train_acc, op=MPI.SUM)
    
    return train_acc / size

In [7]:
#Training Loop:
    
epochs = 5
for epoch in range(epochs):
    train_acc = train(model, x_train, y_train, rank, size)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    test_acc = comm.allreduce(test_acc, op=MPI.SUM)
    
    if rank == 0:
        print(f"Epoch {epoch + 1}: Train accuracy = {train_acc:.4f}, Test accuracy = {test_acc / size:.4f}")


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8766 - loss: 0.4167
1875/1875 - 8s - 4ms/step - accuracy: 0.9717 - loss: 0.1025
313/313 - 2s - 7ms/step - accuracy: 0.9699 - loss: 0.1014
Epoch 1: Train accuracy = 0.9717, Test accuracy = 0.9699
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.9720 - loss: 0.0967
1875/1875 - 9s - 5ms/step - accuracy: 0.9814 - loss: 0.0667
313/313 - 2s - 6ms/step - accuracy: 0.9774 - loss: 0.0738
Epoch 2: Train accuracy = 0.9814, Test accuracy = 0.9774
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.9806 - loss: 0.0654
1875/1875 - 9s - 5ms/step - accuracy: 0.9866 - loss: 0.0474
313/313 - 2s - 7ms/step - accuracy: 0.9809 - loss: 0.0616
Epoch 3: Train accuracy = 0.9866, Test accuracy = 0.9809
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.9849 - loss: 0.0517
1875/1875 - 9s - 5ms/step - accuracy: 0.9898 - loss: 0.0355
313/313 - 2s - 6ms/step - accuracy: 0.9818 - loss: 0.0533
Epoch 4: Train accuracy = 0.9898, Test accura